In [1]:
import pandas as pd
import numpy as np

In [2]:
import tensorflow as tf

In [3]:
from tqdm import *
import math

In [4]:
from keras.datasets import imdb
idx = imdb.get_word_index()

Using TensorFlow backend.


In [5]:
import pickle

In [6]:
idx_arr = sorted(idx, key=idx.get)
idx_arr[:10]

['the', 'and', 'a', 'of', 'to', 'is', 'br', 'in', 'it', 'i']

In [7]:
path = "imdb_full.pkl"
f = open(path, 'rb')
(x_train, labels_train), (x_test, labels_test) = pickle.load(f)

In [8]:
len(x_train), len(x_test)

(25000, 25000)

In [9]:
idx2word = {v: k for k, v in idx.items()}

In [10]:
', '.join(map(str, x_train[0]))

'23022, 309, 6, 3, 1069, 209, 9, 2175, 30, 1, 169, 55, 14, 46, 82, 5869, 41, 393, 110, 138, 14, 5359, 58, 4477, 150, 8, 1, 5032, 5948, 482, 69, 5, 261, 12, 23022, 73935, 2003, 6, 73, 2436, 5, 632, 71, 6, 5359, 1, 25279, 5, 2004, 10471, 1, 5941, 1534, 34, 67, 64, 205, 140, 65, 1232, 63526, 21145, 1, 49265, 4, 1, 223, 901, 29, 3024, 69, 4, 1, 5863, 10, 694, 2, 65, 1534, 51, 10, 216, 1, 387, 8, 60, 3, 1472, 3724, 802, 5, 3521, 177, 1, 393, 10, 1238, 14030, 30, 309, 3, 353, 344, 2989, 143, 130, 5, 7804, 28, 4, 126, 5359, 1472, 2375, 5, 23022, 309, 10, 532, 12, 108, 1470, 4, 58, 556, 101, 12, 23022, 309, 6, 227, 4187, 48, 3, 2237, 12, 9, 215'

In [11]:
' '.join([idx2word[o] for o in x_train[0]])

"bromwell high is a cartoon comedy it ran at the same time as some other programs about school life such as teachers my 35 years in the teaching profession lead me to believe that bromwell high's satire is much closer to reality than is teachers the scramble to survive financially the insightful students who can see right through their pathetic teachers' pomp the pettiness of the whole situation all remind me of the schools i knew and their students when i saw the episode in which a student repeatedly tried to burn down the school i immediately recalled at high a classic line inspector i'm here to sack one of your teachers student welcome to bromwell high i expect that many adults of my age think that bromwell high is far fetched what a pity that it isn't"

In [12]:
labels_train[:10]

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [12]:
vocab_size = 5000

trn = [np.array([i if i<vocab_size-1 else vocab_size-1 for i in s]) for s in x_train]
test = [np.array([i if i<vocab_size-1 else vocab_size-1 for i in s]) for s in x_test]

In [13]:
lens = np.array( [i for i in map(len, trn)] )
print(lens.shape)
(lens.max(), lens.min(), lens.mean())

(25000,)


(2493, 10, 237.71364)

In [14]:
trn = tf.contrib.keras.preprocessing.sequence.pad_sequences( trn , maxlen=500 )
test = tf.contrib.keras.preprocessing.sequence.pad_sequences( test , maxlen=500 )

In [15]:
print(trn.shape)
print(test.shape)

(25000, 500)
(25000, 500)


In [16]:
def run_model(session, predict, loss_val, Xd, yd,
              epochs=1, batch_size=64, print_every=100,
              training=None, plot_losses=False , valx = None , valy = None):

    Xd = np.array(Xd)
    yd = np.array(yd)
  
    correct_prediction = tf.equal( y , predict )
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    train_indicies = np.arange(Xd.shape[0])
    np.random.shuffle(train_indicies)

    training_now = training is not None
    
    variables = [loss_val,correct_prediction,accuracy]
    if training_now:
        variables[-1] = training
    
    # counter 
    iter_cnt = 0
    for e in tqdm_notebook( range(epochs) , leave=False ):
        
        correct = 0
        losses = []
        
        for i in tqdm_notebook( range(int(math.ceil(Xd.shape[0]/batch_size))) , leave=False):
            # generate indicies for the batch
            start_idx = (i*batch_size)%Xd.shape[0]
            idx = train_indicies[start_idx:start_idx+batch_size]
            
            feed_dict = {x: Xd[idx,:],
                         y: yd[idx],
                         is_training: training_now }

            actual_batch_size = yd[idx].shape[0]
            
            loss, corr, _ = session.run(variables,feed_dict=feed_dict)
            
            losses.append(loss*actual_batch_size)
            correct += np.sum(corr)
            
            if training_now and (iter_cnt % print_every) == 0:
                print("Iteration {0}: with minibatch training loss = {1:.3g} and accuracy of {2:.2g}"\
                      .format(iter_cnt,loss,np.sum(corr)/actual_batch_size))
                
            iter_cnt += 1
        total_correct = correct/Xd.shape[0]
        total_loss = np.sum(losses)/Xd.shape[0]
        print("Epoch {2}, Overall loss = {0:.3g} and accuracy of {1:.3g}"\
              .format(total_loss,total_correct,e+1))
        if valx is not None:
            temp1 = session.run(correct_prediction,feed_dict = {x:valx,y:valy,is_training:False})
            print("Validation results here: ",np.sum(temp1)/valx.shape[0])
        
        if plot_losses:
            plt.plot(losses)
            plt.grid(True)
            plt.title('Epoch {} Loss'.format(e+1))
            plt.xlabel('minibatch number')
            plt.ylabel('minibatch loss')
            plt.show()
    return total_loss,total_correct


In [17]:
tf.reset_default_graph()
x = tf.placeholder( tf.int32 , [None , 500 ])
y = tf.placeholder( tf.int32 , [None] )
is_training = tf.placeholder( tf.bool )

emb = tf.get_variable( "emb" , shape = [vocab_size , 32] , dtype=tf.float32 )

emb_lookup = tf.nn.embedding_lookup( params=emb , ids= x)

p1 = tf.reshape( emb_lookup , [-1,500*32] )
p2 = tf.layers.dense( p1, 70 , activation = tf.nn.relu )
p3 = tf.layers.dropout( p2 , .5 )
p4 = tf.layers.dense( p3 , 1 , activation = tf.nn.sigmoid )
p4 = tf.reshape( p4 , [-1] )

pred = tf.cast( p4 >= 0.5 , tf.int32 )

loss = tf.losses.log_loss(labels=y,predictions=p4)
mean_loss = tf.reduce_mean( loss )
train_op = tf.train.AdamOptimizer(0.001).minimize(mean_loss)

with tf.Session() as sess:
    sess.run( tf.global_variables_initializer() )
    run_model( sess,pred,mean_loss,trn,labels_train,epochs=2,training=train_op,valx=test[:500],valy=labels_test[:500] )
    print()

Iteration 0: with minibatch training loss = 0.688 and accuracy of 0.62
Iteration 100: with minibatch training loss = 0.525 and accuracy of 0.8
Iteration 200: with minibatch training loss = 0.256 and accuracy of 0.86
Iteration 300: with minibatch training loss = 0.325 and accuracy of 0.8
Epoch 1, Overall loss = 0.42 and accuracy of 0.781
Validation results here:  0.81


Iteration 400: with minibatch training loss = 0.252 and accuracy of 0.89
Iteration 500: with minibatch training loss = 0.176 and accuracy of 0.92
Iteration 600: with minibatch training loss = 0.123 and accuracy of 0.97
Iteration 700: with minibatch training loss = 0.0701 and accuracy of 0.98
Epoch 2, Overall loss = 0.152 and accuracy of 0.944
Validation results here:  0.878



In [21]:
tf.reset_default_graph()
x = tf.placeholder( tf.int32 , [None , 500 ])
y = tf.placeholder( tf.int32 , [None] )
is_training = tf.placeholder( tf.bool )

emb = tf.get_variable( "emb" , shape = [vocab_size , 32] , dtype=tf.float32 )

emb_lookup = tf.nn.embedding_lookup( params=emb , ids= x)

p1 = tf.layers.dropout( emb_lookup , 0.2 )
p2 = tf.layers.conv1d( p1 , 64 , 5 , activation= tf.nn.relu , padding='same' )
p3 = tf.layers.dropout( p2 , 0.2 )
p4 = tf.layers.max_pooling1d( p3 , 2 , 2 )

p5 = tf.reshape( p4 , [-1,250*64] )
p6 = tf.layers.dense( p5, 100 , activation = tf.nn.relu )
p7 = tf.layers.dropout( p6 , .7 )
p8 = tf.layers.dense( p7 , 1 , activation = tf.nn.sigmoid )
p9 = tf.reshape( p8 , [-1] )

pred = tf.cast( p9 >= 0.5 , tf.int32 )

loss = tf.losses.log_loss(labels=y,predictions=p9)
mean_loss = tf.reduce_mean( loss )
train_op = tf.train.AdamOptimizer(0.01).minimize(mean_loss)

with tf.Session() as sess:
    sess.run( tf.global_variables_initializer() )
    
    run_model( sess,pred,mean_loss,trn,labels_train,epochs=2,
              training=train_op, batch_size = 32 )
    run_model( sess,pred,mean_loss,test,labels_test,epochs=1,batch_size=32)
    
    print()

Iteration 0: with minibatch training loss = 0.691 and accuracy of 0.62
Iteration 100: with minibatch training loss = 0.354 and accuracy of 0.88
Iteration 200: with minibatch training loss = 0.414 and accuracy of 0.81
Iteration 300: with minibatch training loss = 0.414 and accuracy of 0.84
Iteration 400: with minibatch training loss = 0.336 and accuracy of 0.81
Iteration 500: with minibatch training loss = 0.215 and accuracy of 0.88
Iteration 600: with minibatch training loss = 0.439 and accuracy of 0.91
Iteration 700: with minibatch training loss = 0.525 and accuracy of 0.78
Epoch 1, Overall loss = 0.353 and accuracy of 0.849


Iteration 800: with minibatch training loss = 0.125 and accuracy of 0.97
Iteration 900: with minibatch training loss = 0.0647 and accuracy of 0.97
Iteration 1000: with minibatch training loss = 0.186 and accuracy of 0.91
Iteration 1100: with minibatch training loss = 0.485 and accuracy of 0.88
Iteration 1200: with minibatch training loss = 0.313 and accuracy of 0.78
Iteration 1300: with minibatch training loss = 0.232 and accuracy of 0.94
Iteration 1400: with minibatch training loss = 0.185 and accuracy of 0.97
Iteration 1500: with minibatch training loss = 0.0769 and accuracy of 0.97
Epoch 2, Overall loss = 0.242 and accuracy of 0.905


Epoch 1, Overall loss = 0.451 and accuracy of 0.843



In [20]:
tf.reset_default_graph()
x = tf.placeholder( tf.int32 , [None , 500 ])
y = tf.placeholder( tf.int32 , [None] )
is_training = tf.placeholder( tf.bool )

emb = tf.get_variable( "emb" , shape = [vocab_size , 32] , dtype=tf.float32 )
emb_lookup = tf.nn.embedding_lookup( params=emb , ids= x)
data = tf.layers.dropout( emb_lookup , 0.2 )

# lstmCell = tf.contrib.rnn.BasicLSTMCell(150)
# lstmCell_dr = tf.contrib.rnn.DropoutWrapper( cell=lstmCell, output_keep_prob=0.75)

cells = []
for _ in range(3):
    cell = tf.contrib.rnn.GRUCell(64)
    cell = tf.contrib.rnn.DropoutWrapper( cell, output_keep_prob=0.75)
    cells.append(cell)
    
cell = tf.contrib.rnn.MultiRNNCell(cells)
value, _ = tf.nn.dynamic_rnn( cell , data, dtype=tf.float32)

value = tf.transpose( value , [1, 0, 2])

partitions = np.zeros( value.get_shape()[0] )
partitions[-1] = 1

checking = tf.dynamic_partition( value , partitions , 2 )
# something = tf.shape(checking)

last = tf.reshape( checking[1] , [-1,64])
# last = tf.gather(value, int(value.get_shape()[0]) - 1)

p1 = tf.layers.dense( last , 1 , activation=tf.nn.sigmoid )
p2 = tf.reshape( p1 , [-1] )

pred = tf.cast( p2 >= 0.5 , tf.int32 )

loss = tf.losses.log_loss(labels=y,predictions=p2)
mean_loss = tf.reduce_mean( loss )
train_op = tf.train.AdamOptimizer(0.01).minimize(mean_loss)

with tf.Session() as sess:
    sess.run( tf.global_variables_initializer() )
#     a = sess.run( anoter , feed_dict = {x:trn[:100] , y:labels_train[:100] , is_training : True })
#     print(a.shape)
    
    
# #     print(__.shape)
    for i in tqdm_notebook( range(2) ,leave = False ):
        run_model( sess,pred,mean_loss,trn,labels_train,epochs=1,
                  training=train_op, batch_size = 64 )
        run_model( sess,pred,mean_loss,test,labels_test,epochs=1,batch_size=64)
    

Iteration 0: with minibatch training loss = 0.693 and accuracy of 0.59
Iteration 100: with minibatch training loss = 0.418 and accuracy of 0.84
Iteration 200: with minibatch training loss = 0.503 and accuracy of 0.81
Iteration 300: with minibatch training loss = 0.189 and accuracy of 0.92
Epoch 1, Overall loss = 0.41 and accuracy of 0.806


Epoch 1, Overall loss = 0.292 and accuracy of 0.882


Iteration 0: with minibatch training loss = 0.229 and accuracy of 0.89
Iteration 100: with minibatch training loss = 0.154 and accuracy of 0.97
Iteration 200: with minibatch training loss = 0.295 and accuracy of 0.91
Iteration 300: with minibatch training loss = 0.162 and accuracy of 0.94
Epoch 1, Overall loss = 0.223 and accuracy of 0.914


Epoch 1, Overall loss = 0.269 and accuracy of 0.892


##### ^ the encoded context vector gives us the best result as it intuitively should

# 4-th char generation

In [22]:
from keras.utils.data_utils import get_file

In [23]:
text = open("nietzsche.txt").read()
print('corpus length:', len(text))

corpus length: 600901


In [24]:
chars = sorted(list(set(text)))
vocab_size = len(chars)+1
print('total chars:', vocab_size)

total chars: 86


In [25]:
chars.insert(0, "\0")

In [26]:
''.join(chars[1:-6])

'\n !"\'(),-.0123456789:;=?ABCDEFGHIJKLMNOPQRSTUVWXYZ[]_abcdefghijklmnopqrstuvwxyz'

In [27]:
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

In [28]:
idx = [char_indices[c] for c in text]

In [29]:
idx[:10]

[40, 42, 29, 30, 25, 27, 29, 1, 1, 1]

In [30]:
''.join( [indices_char[i] for i in idx[:70]] )

'PREFACE\n\n\nSUPPOSING that Truth is a woman--what then? Is there not gro'

In [31]:
cs=3
c1_dat = [idx[i] for i in range(0, len(idx)-1-cs)]
c2_dat = [idx[i+1] for i in range(0, len(idx)-1-cs)]
c3_dat = [idx[i+2] for i in range(0, len(idx)-1-cs)]
c4_dat = [idx[i+3] for i in range(0, len(idx)-1-cs)]

In [32]:
x1 = np.array(c1_dat)
x2 = np.array(c2_dat)
x3 = np.array(c3_dat)

y = np.array(c4_dat)
x1.shape , x2.shape , x3.shape , y.shape

((600897,), (600897,), (600897,), (600897,))

In [33]:
n_fac = 32

In [34]:
tf.reset_default_graph()

em_matrix = tf.get_variable( "em_matrix" , shape = [vocab_size,n_fac] , dtype=tf.float32 )

in1 = tf.placeholder( tf.int32 , shape = [None] )
in2 = tf.placeholder( tf.int32 , shape = [None] )
in3 = tf.placeholder( tf.int32 , shape = [None] )

out = tf.placeholder( tf.int64 , shape = [None] )

i1 = tf.nn.embedding_lookup( em_matrix , in1)
i2 = tf.nn.embedding_lookup( em_matrix , in2)
i3 = tf.nn.embedding_lookup( em_matrix , in3)

is_training = tf.placeholder( tf.bool )

h1 = tf.layers.dense( i1 , 20 , activation=tf.nn.relu , name = "in_hidden" )
t1 = tf.layers.dense( i2 , 20 , activation=tf.nn.relu , name = "in_hidden" , reuse=True)
t2 = tf.layers.dense( i3 , 20 , activation=tf.nn.relu , name = "in_hidden" , reuse=True)

t3 = tf.layers.dense( h1 , 20 , activation=tf.nn.tanh , name = "hidden_hidden" ) + t1
t4 = tf.layers.dense( t3 , 20 , activation=tf.nn.tanh , name = "hidden_hidden" , reuse=True ) + t2

t5 = tf.layers.dense( t4 , vocab_size , name = "hidden_out" )

loss = tf.losses.softmax_cross_entropy( tf.one_hot(out,vocab_size) , t5 )
loss = tf.reduce_mean(loss)

train_op = tf.train.AdamOptimizer(0.001).minimize(loss)

correct_index = tf.argmax(t5,axis=1)
correct = tf.reduce_sum( tf.cast( tf.equal( correct_index , out ) , dtype=tf.float32 ) )

In [36]:
def run_model(q1,q2,q3,q4,session,l,t,c,saver,batch_size=64,epoch=5):
    
    train_indices = np.arange( q1.shape[0] )
    np.random.shuffle( train_indices )
    
    for e in tqdm_notebook( range(epoch) ):
        
        correct = 0
        losses = []
        for cc in tqdm_notebook( range(int(math.ceil(q1.shape[0]/batch_size))) ):
            
            start_idx = (cc*batch_size)%q1.shape[0]
            idx = train_indices[ start_idx : start_idx+batch_size]
            
            feed_dict = { in1:q1[idx] , in2:q2[idx] , in3:q3[idx] , out:q4[idx] , is_training: True } 
            actual_batch_size = q1[idx].shape[0]
            
            lr, corr, _ = session.run( [l,c,t] , feed_dict=feed_dict )
            losses.append(lr*actual_batch_size)
            correct += np.sum(corr)
        
        saver.save(sess, "tmp/first_rnn",global_step = e)
        total_correct = correct/q1.shape[0]
        total_loss = np.sum(losses)/q1.shape[0]
        print("Epoch {2}, Overall loss = {0:.3g} and accuracy of {1:.3g}".format(total_loss,total_correct,e+1))

In [38]:
saver = tf.train.Saver()
with tf.Session() as sess:
    tf.global_variables_initializer().run()
#     saver.restore(sess, "tmp/first_rnn-5")
#     for i in range(2):
#         _,__,lo = sess.run( [train_op,correct,loss] ,  feed_dict={ in1:x1[:1000] , in2:x2[:1000] , in3:x3[:1000] , out:y[:1000] , is_training: True } )
#         print( __,lo )
    run_model(x1,x2,x3,y,sess,loss,train_op,correct,saver,batch_size=64,epoch=3)
#     prediction("men",sess)


Epoch 1, Overall loss = 2.43 and accuracy of 0.315


Epoch 2, Overall loss = 2.29 and accuracy of 0.343


Epoch 3, Overall loss = 2.26 and accuracy of 0.35



In [32]:
def prediction(te,sess):
    temp = [ char_indices[i] for i in te ]
    tep = sess.run( correct_index , feed_dict = { in1 : temp[:1] ,in2 : temp[1:2] , in3 : temp[2:3] ,is_training:False } )
    print( indices_char[tep[0] ] )

In [27]:
[char_indices[i] for i in "Preface" ]

[40, 71, 58, 59, 54, 56, 58]

In [131]:
tf.reset_default_graph()

em_matrix = tf.get_variable( "em_matrix" , shape = [vocab_size,n_fac] , dtype=tf.float32 )

in1 = tf.placeholder( tf.int64 , shape = [None,8] )
out = tf.placeholder( tf.int64 , shape = [None] )

i1 = tf.nn.embedding_lookup( em_matrix , in1)

is_training = tf.placeholder( tf.bool )

cell = tf.contrib.rnn.BasicLSTMCell(13)
value,_ = tf.nn.dynamic_rnn(cell,i1,dtype=tf.float32)

value = tf.transpose( value , [1, 0, 2])

partitions = np.zeros( value.get_shape()[0] )
partitions[-1] = 1

part = tf.dynamic_partition( value , partitions , 2 )

sque = tf.squeeze(part[1],axis=[0])
t5 = tf.layers.dense( sque , vocab_size )

loss = tf.losses.softmax_cross_entropy( tf.one_hot(out,vocab_size) , t5 )
loss = tf.reduce_mean(loss)

train_op = tf.train.AdamOptimizer(0.001).minimize(loss)

correct_index = tf.argmax(t5,axis=1)
correct = tf.reduce_sum( tf.cast( tf.equal( correct_index , out ) , dtype=tf.float32 ) )

In [142]:
saver = tf.train.Saver()
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    saver.restore(sess, "tmp/second_rnn-5")
#     for i in range(10):
#         _,__,lo = sess.run( [train_op,correct,loss] ,  feed_dict={ in1:X[:1000], out:y[:1000] , is_training: True } )
#         print( __,lo )
#     run_model_2(X,y,sess,loss,train_op,correct,saver,batch_size=256,epoch=10)
    prediction_2( 'it is no',sess)
#     _ = sess.run(sque,feed_dict={in1:X[:12],out:y[:12]})
#     print( _.shape )

INFO:tensorflow:Restoring parameters from tmp/second_rnn-5
[73]
t


In [79]:
cs=8

In [85]:
c_in_dat = [[idx[i+n] for i in range(0, len(idx)-1-cs)] for n in range(cs)]

In [86]:
np.array( c_in_dat ).shape

(8, 600892)

In [92]:
c_out_dat = [idx[i+cs] for i in range(0, len(idx)-1-cs)]

In [93]:
X = np.array( c_in_dat).T
y = np.array( c_out_dat).T

In [94]:
X.shape , y.shape

((600892, 8), (600892,))

In [129]:
def run_model_2(q1,q4,session,l,t,c,saver,batch_size=64,epoch=5):
    
    train_indices = np.arange( q1.shape[0] )
    np.random.shuffle( train_indices )
    
    for e in tqdm_notebook( range(epoch) ):
        
        correct = 0
        losses = []
        for cc in tqdm_notebook( range(int(math.ceil(q1.shape[0]/batch_size))) ):
            
            start_idx = (cc*batch_size)%q1.shape[0]
            idx = train_indices[ start_idx : start_idx+batch_size]
            
            feed_dict = { in1:q1[idx] , out:q4[idx] , is_training: True } 
            actual_batch_size = q1[idx].shape[0]
            
            lr, corr, _ = session.run( [l,c,t] , feed_dict=feed_dict )
            losses.append(lr*actual_batch_size)
            correct += np.sum(corr)
        
        saver.save(sess, "tmp/second_rnn",global_step = e)
        total_correct = correct/q1.shape[0]
        total_loss = np.sum(losses)/q1.shape[0]
        print("Epoch {2}, Overall loss = {0:.3g} and accuracy of {1:.3g}".format(total_loss,total_correct,e+1))

In [136]:
def prediction_2(te,sess):
    temp = [ char_indices[i] for i in te ]
    tep = sess.run( correct_index , feed_dict = { in1 : [temp] ,is_training:False } )
    print(tep)
    print( indices_char[tep[0] ] )

In [170]:
c_out_dat = [[idx[i+n] for i in range(1, len(idx)-cs)] for n in range(cs)]

In [171]:
y = np.array( c_out_dat).T

In [172]:
y.shape

(600892, 8)

In [288]:
tf.reset_default_graph()
tf.set_random_seed(1)
em_matrix = tf.get_variable( "em_matrix" , shape = [vocab_size,n_fac] , dtype=tf.float32 )

in1 = tf.placeholder( tf.int64 , shape = [None,8] )
out = tf.placeholder( tf.int64 , shape = [None,8] )

i1 = tf.nn.embedding_lookup( em_matrix , in1)

is_training = tf.placeholder( tf.bool )

tf.nn.rnn_cell()

cell = tf.contrib.rnn.BasicLSTMCell(13)

value , _ = tf.nn.dynamic_rnn(cell,i1,dtype=tf.float32)

t5 = tf.layers.dense( value , vocab_size )

loss = tf.losses.softmax_cross_entropy( tf.one_hot(out,vocab_size) , t5 )

train_op = tf.train.AdamOptimizer(0.01).minimize(loss)

correct_index = tf.argmax(t5,axis=2)
correct = tf.reduce_sum( tf.cast( tf.equal( correct_index , out ) , dtype=tf.float32 ) ) / 8.0

In [231]:
def run_model_3(q1,q4,session,l,t,c,saver,batch_size=64,epoch=5):
    
    train_indices = np.arange( q1.shape[0] )
#     np.random.shuffle( train_indices )
    
    for e in tqdm_notebook( range(epoch) ):
        
        correct = 0
        losses = []
        for cc in tqdm_notebook( range(int(math.ceil(q1.shape[0]/batch_size))) ):
            
            start_idx = (cc*batch_size)%q1.shape[0]
            idx = train_indices[ start_idx : start_idx+batch_size]
            
            feed_dict = { in1:q1[idx] , out:q4[idx] , is_training: True } 
            actual_batch_size = q1[idx].shape[0]
            
            lr, corr, tr_op  = session.run( [l,c,t] , feed_dict=feed_dict )
            losses.append(lr*actual_batch_size)
            correct += corr
        
        saver.save(sess, "tmp/third_rnn",global_step = e)
        total_correct = correct/q1.shape[0]
        total_loss = np.sum(losses)/q1.shape[0]
        print("Epoch {2}, Overall loss = {0:.3g} and accuracy of {1:.3g}".format(total_loss,total_correct,e+1))

In [222]:
def prediction_3(te,sess):
    print( list(te) )
    temp = [ char_indices[i] for i in te ]
    tep = sess.run( correct_index , feed_dict = { in1 : [temp] ,is_training:False } )
    print( [indices_char[j] for j in tep[0]] )

In [232]:
saver = tf.train.Saver()
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    saver.restore(sess, "tmp/third_rnn-4")
#     for i in range(10):
#         _,__,lo = sess.run( [train_op,correct,loss] ,  feed_dict={ in1:X[:1000], out:y[:1000] , is_training: True } )
#         print( __,lo )
    run_model_3(X,y,sess,loss,train_op,correct,saver,batch_size=64,epoch=1)
#     prediction_3( ' this is',sess)
#     _ = sess.run(sque,feed_dict={in1:X[:12],out:y[:12]})
#     print( _.shape )

INFO:tensorflow:Restoring parameters from tmp/third_rnn-4


Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.



Epoch 1, Overall loss = 2.17 and accuracy of 0.37

